## Recurrent Neural Networks - Supervised Learning II - MDS Computational Linguistics

### Goal of this tutorial
- Introduce Recurrent Neural Networks (RNNs)
- Implement RNN for sentiment analysis
- Implement Long-Short Term Memories (LSTMs) for sentiment analysis
- Implement Gated Recurrent Units (GRUs) for sentiment analysis

### General
- This notebook was last tested on Python 3.6.9 and PyTorch 1.2.0

We would like to acknowledge the following materials which helped as a reference in preparing this tutorial:
- https://github.com/UBC-NLP/dlnlp2019/blob/master/slides/RNN.pdf

In [6]:
import pandas as pd
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
import torch
import torch.nn as nn

### Embedding Layer

The ``Embedding`` layer in Pytorch is where we pass our vocabulary to get get back a word vector for each word in the vocabulary. We need to know what the input and output of this layer look like. Let's first
do this on a dummy example where we have two sentences ``x_1`` and ``x_2``. Let's assume we have the two sentences as:

In [7]:
x_1 = "He is very nice"
x_2 = "She is very kind"

Let's convert the two sentences into indexes (each word is replaced with its index in the vocabulary).
Let's assume our ``vocabulary size`` is set to 100. Remember, vocabulary size is a hyper-parameter.
Let's also store that ``vocabulary size`` in a variable ``VOCAB_SIZE`` now as we will need to pass it to the ``Embedding`` layer later.

In [8]:
x_1 = [1, 25, 40, 5]
x_2 = [4, 25, 40, 99]
VOCAB_SIZE = 100

### Max sequence length

One last thing we need to think about is the ``length of each sequence``. The two examples above are nicely set to equal length = 4. This does not need to be the case, as we can have sequences of varying lengths. We will be passing a batch of sentences to Pytorch and the max sequence length will be set to the length of the longest sentence in that batch. The rest of sentences (shorter ones) will be padded with zeros. Now, do we need to explicitly provide the max sequence length to Pytorch? And how do we know the max seq length for each batch, if different batches have sequences of varying lengths and each batch is set to the max sent in that batch? Well, rest assured, we don't really need to worry about that. Pytorch will assign a max seq length for each batch. We will be able to inspect the max seq length for a given batch using output of the ``Embedding`` layer. (We will see that soon).

## Size of word vector

The ``Embedding`` layer will give us a vector for each word in the vocabulary.
Now, we will need to tell it what size we want for that vector. Popular values for a vector size are usally between 100-300 for many tasks (e.g., sentiment analysis"). Let's set it to 200 dimensions. (You are encouraged to play with this value as practice). All words in the vocabulary will have the same embedding size. Let's put that hyper-parameter in a variable ``WORD_VEC_SIZE``:

In [9]:
WORD_VEC_SIZE= 300

We are now ready to call the ``Embedding`` class to construct an embeddings tensor:

In [10]:
# Constructing an embedding Layer:
embedding = nn.Embedding(VOCAB_SIZE, WORD_VEC_SIZE)
input = torch.LongTensor([ x_1, x_2 ])
embedded=embedding(input)
print(embedded)

tensor([[[ 0.2781, -0.9621, -0.4223,  ...,  1.6861, -0.3435,  0.7307],
         [ 2.1454,  0.4632,  0.3452,  ...,  0.5029,  1.4000,  0.1168],
         [-3.6817,  2.1989,  0.8434,  ...,  0.2572,  0.6342, -0.4037],
         [ 0.6670, -0.0886, -0.7938,  ..., -1.1525,  0.3251,  0.3102]],

        [[ 0.5600,  1.7931,  0.0619,  ..., -0.6966,  0.9701,  1.4421],
         [ 2.1454,  0.4632,  0.3452,  ...,  0.5029,  1.4000,  0.1168],
         [-3.6817,  2.1989,  0.8434,  ...,  0.2572,  0.6342, -0.4037],
         [-0.8286,  0.9023, -0.7984,  ...,  1.1266,  0.0614,  0.5058]]],
       grad_fn=<EmbeddingBackward>)


**Let's print the shape of this tensor:**

In [11]:
print(embedded.size())

torch.Size([2, 4, 300])


This is telling us:
- We have ``2 examples`` (that is, our ``x_1`` and ``x_2``). (Note: We will be paqssing a whole batch to the ``Embedding`` class and so this first dimension will be equal to the ``batch size``.
- For each of the two examples, we have a ``max sequence length`` = 4 (x_1 and x_2 each had 4 indexes).
- The ``word vector dimension`` is set to 300.

### Max sequence length: Another note

Recall from above we mentioned Pytorch automatically infers the max sequence length for each batch. 
For the example above (as you can see from the second dimension returned by ``embedded.size()``, Pytorch 
inferred the max seq length for this batch of two sentences is 4.
Let's just adjust the second example, adding two more words (the string "and kind"). Note, both our ``VOCAB_SIZE`` and
``WORD_VEC_SIZE`` stay the same as before. We assign the word "and" an index of "7" and the word "considerate" and index of "60". Note that we have to pad the first example ``x_1`` with zeros in the end. (Try removing the zero padding. What do you observe when you run your code with the ``Embedding`` class? Hint: You will get an error.):

In [12]:
x_1 = "He is very nice"
x_2 = "She is very kind and considerate"
x_1 = [1, 25, 40, 5, 0, 0]
x_2 = [4, 25, 40, 99, 7, 60]

Now, let's create a new ``embedding`` layer by creating a new instance of the ``Embedding`` class:

In [13]:
# Constructing an embedding Layer:
embedding = nn.Embedding(VOCAB_SIZE, WORD_VEC_SIZE)
input = torch.LongTensor([ x_1, x_2 ])
embedded=embedding(input)
print(embedded)

tensor([[[-7.5832e-01,  6.8376e-01, -5.7114e-01,  ...,  9.2972e-01,
          -1.6073e-03,  3.4804e-01],
         [-8.4253e-01, -1.5546e+00, -5.4597e-02,  ...,  7.9151e-01,
          -2.3702e+00,  4.0472e-01],
         [ 5.2497e-01, -9.5013e-01, -4.2413e-01,  ..., -4.1445e-01,
           2.5005e-01,  2.1687e-01],
         [-3.0029e-03,  1.2672e+00,  3.9032e-02,  ...,  2.3105e-01,
           2.5055e+00, -4.1138e-01],
         [-1.5398e+00, -9.5563e-01, -2.0218e-01,  ...,  3.3748e-01,
          -1.1845e+00,  1.4116e-01],
         [-1.5398e+00, -9.5563e-01, -2.0218e-01,  ...,  3.3748e-01,
          -1.1845e+00,  1.4116e-01]],

        [[-4.2902e-01,  2.6079e-01,  2.2463e+00,  ...,  1.0946e-02,
          -3.4559e-01, -6.9403e-01],
         [-8.4253e-01, -1.5546e+00, -5.4597e-02,  ...,  7.9151e-01,
          -2.3702e+00,  4.0472e-01],
         [ 5.2497e-01, -9.5013e-01, -4.2413e-01,  ..., -4.1445e-01,
           2.5005e-01,  2.1687e-01],
         [ 6.1487e-01, -1.8579e-01, -4.9036e-01,  ...

**If we inspect the shape of the new tensor ``embedded``, we will see the second dimension now changed to 6, to match the max sequence length:**

In [14]:
print(embedded.size())

torch.Size([2, 6, 300])


### How does Pytorch initialize word vector dimensions/weights?

Note that Pytorch initializes the word vectors with initialized from a normal distribution $ \mathcal{N}(0, 1) $. The word embedding weights are by default learnable parameters in Pytorch and so they will be adjusted during training. (Note: These weights can be initialized from an external word embedding tool such as Word2vec, Fasttext, or Glove. Also, the weights can be frozen, which is a reasonable option when initialized from an external tool. You can choose to keep learning them within the model with your training data). Below we show the ones initialized from a normal distribution by Pytorch.

In [15]:
embedding.weight

Parameter containing:
tensor([[-1.5398e+00, -9.5563e-01, -2.0218e-01,  ...,  3.3748e-01,
         -1.1845e+00,  1.4116e-01],
        [-7.5832e-01,  6.8376e-01, -5.7114e-01,  ...,  9.2972e-01,
         -1.6073e-03,  3.4804e-01],
        [ 2.9145e-01, -3.5757e-01, -1.3327e+00,  ...,  4.5862e-01,
         -1.7158e-01,  3.4948e-01],
        ...,
        [-9.1737e-01, -4.9251e-03,  8.9590e-01,  ..., -1.3764e+00,
          2.7926e+00, -2.5609e-01],
        [ 2.8130e-01,  2.6243e-01, -1.5968e+00,  ...,  8.2734e-02,
         -7.7934e-01, -7.5185e-01],
        [ 6.1487e-01, -1.8579e-01, -4.9036e-01,  ..., -9.8516e-01,
          4.3174e-02, -3.8600e-01]], requires_grad=True)

More information about the ``Embedding`` class can be found [here](https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html#Embedding).

## Recurrent Neural Networks
Recurrent Neural Networks (RNNs) are used to model sequences of arbitrary length (e.g., sequence of words in a sentence, sequence of sentences in a document, sequence of frames in a video). RNNs typically use their internal state (memory) to process sequence of inputs. At each time-step, RNNs output a prediction and hidden state, feeding its previous hidden state into each next step. RNNs are applied in a wide range of NLP applications:
- language modeling, where RNN can condition on **all** previous words in the corpus unlike n-gram language model
- text classification, where the states act as features (we will see sentiment analysis in this tutorial)
- machine translation, where a RNN is used to process a sentence in source language and another RNN is used to decode the sentence in target language (we will see this in the "Machine Translation" course)
- sequence labeling, where the states in RNN are used to predict a category for each item in the sequence (we might see named entity recognition in the next tutorial)

Recommended reading for understanding the theory of RNNs: https://github.com/UBC-NLP/dlnlp2019/blob/master/slides/RNN.pdf 


### Grabbing few tweets using torchtext

Let us follow **torchtext** tutorial to read few tweets from the [sentiment analysis dataset](http://alt.qcri.org/semeval2016/task4/) used in the previous tutorial on feedforward neural networks. The preprocessed (tokenization, removing URLs, mentions, hashtags and so on) tweets are placed under ``data/sentiment-twitter-2016-task4`` folder in three files as ``train.tsv``, ``dev.tsv`` and ``test.tsv``.  

Let us view few tweets from ``train.tsv`` using pandas.

In [68]:
import pandas as pd
df = pd.read_csv("./data/sentiment-twitter-2016-task4/train.tsv", sep = '\t', header=None, names=['tweet','label']) # the separator of tsv file is `\t`
df.head()

,tweet,label
0,dear <<<MENTION>>> the newooffice for mac is g...,2
1,<<<MENTION>>> how about you make a system that...,2
2,i may be ignorant on this issue but should we ...,2
3,thanks to <<<MENTION>>> i just may be switchin...,2
4,if i make a game as a <<<HASHTAG>>> universal ...,0


**We import the relevant packages, define the tokenizer and TorchText's fields.**

In [69]:
# import related packages
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset

# define the white space tokenizer to get tokens
def tokenize_en(tweet):
    """
    Tokenizes English tweet from a string into a list of strings (tokens)
    """
    return tweet.strip().split()

# define the TorchText's fields
TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
LABEL = Field(sequential=False, unk_token=None)

**To use the different splits (training, development and testing), we use `TabularDataset` class to load datasets.**

In [70]:
train, val, test = TabularDataset.splits(
    path="./data/sentiment-twitter-2016-task4/", # the root directory where the data lies
    train='train.tsv', validation="dev.tsv", test="test.tsv", # file names
    format='tsv',
    skip_header=False, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
    fields=[('tweet', TEXT), ('label', LABEL)])

**Build our vocabulary to map words to integers.**

In [71]:
TEXT.build_vocab(train, min_freq=3) # builds vocabulary based on all the words that occur at least twice in the training set
LABEL.build_vocab(train)

**Initialize the iterators for the train, validation, and test data. Note that we set ``sort`` as False so as to not sort examples based on similar lengths which minimizes padding.**

In [72]:
from torchtext.data import Iterator, BucketIterator

train_iter, val_iter, test_iter = BucketIterator.splits(
 (train, val, test), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(4,64,64),
 sort_key=lambda x: len(x.tweet), 
 sort=False,
# A key to use for sorting examples in order to batch together examples with similar lengths and minimize padding. 
 sort_within_batch=False
)

**Create a batch of four examples and print them**

In [73]:
# create a single batch and terminate the loop
for batch in train_iter:
    tweets = batch.tweet
    labels = batch.label
    break  #we use first batch as an example.

# print the four examples with padding 
print("processed tweets: ")
for j in range(tweets.shape[1]): # sample loop
    tmp = []
    for i in range(tweets.shape[0]): # token loop
        tmp.append(TEXT.vocab.itos[tweets[i,j]])
    print(j," sample:",tmp)

processed tweets: 
0  sample: ['hillary', 'and', 'chelsea', 'are', 'beautiful', 'women', 'may', 'god', 'continue', 'to', 'bless', 'you', 'president', 'clinton', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
1  sample: ['i', 'just', 'wanna', 'say', "i'm", 'so', 'happy', 'proud', 'that', 'justin', 'got', 'his', '<<<hashtag>>>', 'debut', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
2  sample: ['i', 'just', 'realized', "i'm", 'seeing', 'tomorrow', 'holy', 'shit', "i've", 'waited', 'nearly', 'a', 'year', 'for', 'this', '<pad>', '<pad>', '<pad>', '<pad>']
3  sample: ['going', 'to', 'see', 'magic', 'mike', 'xxl', 'may', 'be', 'the', 'single', 'best', 'decision', 'i', 'have', 'ever', 'made', 'in', 'my', 'life']


**Now we set ``sort`` as True so as to sort examples based on similar lengths which minimizes padding.**

**Let us initialize the new iterators for the train, validation, and test data.**

**Let us pick up 4 tweets from the training set and convert them to tensors.**

**Create a batch of four examples and print them**

In [74]:
# create a single batch and terminate the loop
for batch in train_iter:
    tweets = batch.tweet
    labels = batch.label
    break  #we use first batch as an example.

# print the four examples with padding 
print("processed tweets: ")
for j in range(tweets.shape[1]): # sample loop
    tmp = []
    for i in range(tweets.shape[0]): # token loop
        tmp.append(TEXT.vocab.itos[tweets[i,j]])
    print(j," sample:",tmp)

processed tweets: 
0  sample: ['joe', 'biden', "here's", 'my', 'thinking', 'on', 'the', '<<<digit>>>', 'race', '<<<url>>>', "c'mon", 'joe', 'run', 'we', 'need', 'the', '<unk>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
1  sample: ["federer's", 'serve', 'is', 'just', 'perfect', '<<<hashtag>>>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
2  sample: ['hold', 'on', 'partner', 'i', 'got', 'bars', 'to', 'and', 'they', 'kinda', 'scary', "i'm", 'about', 'to', 'show', 'you', 'how', 'may', '<unk>', 'this', 'mariah', "carey's"]
3  sample: ['great', 'scott', 'lexus', 'reveals', 'hoverboard', 'but', 'you', "can't", 'buy', 'it', 'lexus', 'revealed', 'on', 'tuesday', 'a', '<<<digit>>>', 'second', '<unk>', 'that', '<<<url>>>', '<pad>', '<pad>']


### Creating a single hidden layer RNN

PyTorch has ``torch.nn.RNN`` module that implements the vanilla (Elman) RNN with *tanh* or *ReLU* non-linearity. The documentation for this module is [here](https://pytorch.org/docs/stable/nn.html#torch.nn.RNN). Let us use the sample batch of five examples created before to understand this module.

In this tutorial, we will represent the input tweet using a sequence of word embeddings (for each word present in the tweet). We will use ``torch.nn.Embedding module`` to store word vectors corresponding to words in the vocabulary.

Before implementing the embedding module for our usecase, let us compute the size of the word vocabulary.

In [23]:
VOCAB_SIZE = len(TEXT.vocab.stoi)
print(VOCAB_SIZE)

4876


Let us implement the embedding module (whose underlying weight matrix shape is (``vocabulary size`` $\times$ ``word embedding size``) for our usecase:

In [24]:
# an Embedding module containing 10 dimensional tensor for each word in the vocabulary
import torch
import torch.nn as nn
WORD_VEC_SIZE=300
# Note, the parameters to Embedding class below are:
# num_embeddings (int): size of the dictionary of embeddings
# embedding_dim (int): the size of each embedding vector
# For more details on Embedding class, see: https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/sparse.py
embedding = nn.Embedding(VOCAB_SIZE, WORD_VEC_SIZE, sparse=True)

Let us now feed the tensors of our sample batch to the embedding module and extract the sequence of word embeddings for each tweet.

In [25]:
# print tensor containing word ids for our batch
print("*"*50, "\n Word ids for the first batch (recall, it has 4 sentences, each column representing a sentence): \n", tweets.data, "\n","*"*50,)

# feed the "word ids" tensor to the embedding module
tweet_input_embeddings = embedding(tweets)

# print the dimensions of the tweet_embeddings
print("*"*50, "\n Tweet input word embeddings size: ", tweet_input_embeddings.size(), "\n","*"*50,) 
# first dimension - sequence length: number of words per example (same across the whole batch, after padding) --> max_seq = 22
# second dimension -  batch size / number of examples in the batch --> 4
# third dimension - number of dimensions in the word vector

************************************************** 
 Word ids for the first batch (recall, it has 5 sentences): 
 tensor([[ 139,  936,  670,  140],
        [ 130,  737,    8, 3160],
        [ 467,   14, 2001,  128],
        [  23,   28,    6, 3139],
        [ 530,  544,  105,  726],
        [   8,   11, 2655,   26],
        [   2,    1,    3,   17],
        [  13,    1,    9,   98],
        [ 621,    1,   81,  323],
        [   5,    1,  947,   20],
        [ 354,    1, 2502,  128],
        [ 139,    1,   37, 1359],
        [ 246,    1,   40,    8],
        [  54,    1,    3,  118],
        [ 194,    1,  114,    7],
        [   2,    1,   17,   13],
        [   0,    1,  104,  578],
        [   1,    1,   16, 4686],
        [   1,    1,    0,   25],
        [   1,    1,   27,    5],
        [   1,    1,  178,    1],
        [   1,    1, 1080,    1]]) 
 **************************************************
************************************************** 
 Tweet input word embeddings siz

Let's actually view the actual word embeddings tensor for this batch:

In [26]:
print("*"*50, "\n Embeddings for the first batch: ", tweet_input_embeddings, "\n","*"*50,) 

************************************************** 
 Embeddings for the first batch:  tensor([[[-6.2361e-01, -1.6901e-01, -6.3566e-02,  ...,  5.4636e-01,
          -8.6838e-01,  1.8328e+00],
         [-2.4383e+00,  4.0745e-02,  1.2797e+00,  ..., -2.6321e-01,
           1.8393e+00, -1.1585e-02],
         [-3.1749e-01,  1.1447e-01,  9.6525e-02,  ...,  1.6828e+00,
          -5.3543e-01, -1.4563e+00],
         [-2.0944e-01, -7.5421e-01, -7.1625e-01,  ..., -3.5963e-02,
          -6.7576e-01,  1.1171e+00]],

        [[-7.0526e-01,  4.4238e-01, -6.9463e-01,  ...,  5.7985e-01,
           1.3078e+00,  7.1315e-02],
         [-1.4456e+00,  6.9684e-01, -7.5271e-01,  ...,  1.5451e-01,
           1.7786e+00, -6.0696e-01],
         [ 4.4490e-01,  8.0471e-01, -5.7391e-01,  ..., -9.2677e-01,
           8.1939e-01, -1.2917e-01],
         [-6.7699e-01,  4.4199e-01,  9.6667e-01,  ..., -8.2689e-01,
          -5.5547e-01,  4.1825e-01]],

        [[-1.0012e+00, -5.6612e-01, -9.6331e-01,  ..., -7.7925e-01,
  

What we are seeing is the actual word vectors representing each of the 4 sentences (i.e., whole batch).
This is dimension 2 in ``tweet_input_embeddings``. 

In [27]:
tweet_input_embeddings.size()[1]

4

As mentioned, ``max_seq length`` for this batch is ``20``, which is dimension 1 (indexed as 0 in Pytorch, similar to Python) 
in ``tweet_input_embeddings``: 

In [28]:
tweet_input_embeddings.size()[0]

22

Now, dimension 3 in ``tweet_input_embeddings`` (indexed as 2) is the size of the word vectors:

In [29]:
tweet_input_embeddings.size()[2]

300

Let's look at the vector for the ``first word`` in the ``first sentence`` in the batch:

In [30]:
len(tweet_input_embeddings[:1, :1, :])

1

In [31]:
tweet_input_embeddings[:1, :1, :]

tensor([[[-0.6236, -0.1690, -0.0636, -1.4055,  0.4290,  1.0382,  0.5280,
          -0.3858,  0.2720, -0.4258, -0.9499,  0.1718, -2.0671, -0.2302,
          -1.8174,  1.2515,  0.3675, -0.0127, -1.3980,  0.4798,  1.5405,
          -0.0712,  0.6253,  1.6236,  1.5369, -0.6570,  3.1210,  1.2583,
           0.1715,  2.4443,  0.6548, -0.6424, -0.9829, -0.1534, -0.2138,
          -1.1182, -0.0069, -0.7067,  0.7984,  1.7900, -0.2504, -3.6824,
          -0.6083, -0.8782,  1.5293,  0.2707, -1.1001, -0.9750,  1.2913,
           1.7947,  0.8527, -1.7923, -0.5214, -0.1633,  1.6047, -0.0151,
          -0.6170,  0.5219, -0.6206, -0.8031,  0.1429,  0.5681,  1.0633,
          -1.1100,  0.7163,  1.0081, -0.1245, -0.5448, -0.3039,  0.7133,
          -0.1567, -0.4496,  0.4882, -0.1852,  0.2290,  1.8149, -0.6632,
           0.6523, -0.9498,  0.8099, -0.0243, -2.8415,  1.0371, -1.5053,
          -0.1361, -0.0725,  2.5818,  2.3706, -0.3516,  0.6985,  0.0395,
           0.8538, -1.4871, -1.1573, -0.4045,  0.40

Let's look at the ``first 5 dimensions`` of that same ``first word`` of the ``first sentence``:

In [32]:
tweet_input_embeddings[:1, :1, :5]

tensor([[[-0.6236, -0.1690, -0.0636, -1.4055,  0.4290]]],
       grad_fn=<SliceBackward>)

The following shows you the ``first 5 dimensions`` of the ``first word`` from ``each of the 4 sentences``

In [33]:
tweet_input_embeddings[:1, :, :5]

tensor([[[-0.6236, -0.1690, -0.0636, -1.4055,  0.4290],
         [-2.4383,  0.0407,  1.2797, -1.0156, -1.4771],
         [-0.3175,  0.1145,  0.0965, -0.9208, -1.6422],
         [-0.2094, -0.7542, -0.7162,  2.7994,  0.2884]]],
       grad_fn=<SliceBackward>)

The following shows you the ``last 7 dimensions`` of the ``last word`` from ``the last sentence``. Enjoy!

In [34]:
tweet_input_embeddings[-1:, 3:, -7:]

tensor([[[-1.4670,  0.7937,  0.4700,  0.1783, -1.0202, -0.0371,  2.5147]]],
       grad_fn=<SliceBackward>)

We will be passing the embedding vector for the first word, Simultaneously for each senetence in the batch, to the RNN. But let's now define an RNN module first:

In [35]:
"""
define the RNN module
"""
# first input - number of dimesnions for word vectors for a vector x (300, size of the word embedding)
# second input - number of nodes in hidden state h_t (50, size of the hidden layer)
# third input - number of recurrent layers (we set it to 1)
rnn = nn.RNN(input_size=300, hidden_size=50, num_layers=1) # input_size, hidden_size, num_layers

We will now pass the ``tweet_input_embeddings`` (representations of words in our batch) to RNN. Before we do, we need to know RNN also *optionally* takes a parameter for the ``initial hidden state h0`` (that is, the hidden state we will input to the model before it runs. It is just something that we need to start the model. If we don't provide it, Pytorch will just initialize h0 to a tensor of zeros. 

Let's construct an ``initial hidden state h0``. Note the shape of its tensor, and what each of the 3 parameters it takes mean.

In [36]:
"""
hidden layer at time-step 0 (h_0)
"""
# first dimension - number of RNN layers (1)
# second dimension - number of examples/sentences in a batch
# third dimension - number of nodes in hidden layer (50, size of the hidden layer)
h0 = torch.randn(1, 4, 50)
print("The shape as as expected: ", h0.shape)

The shape as as expected:  torch.Size([1, 4, 50])


Let us feed both the hidden representation constructed above and tweet embeddings to our RNN model.
We will get back two objects ``output`` and ``hn`` that we will need to understand.

In [37]:
"""
forward propagation over the RNN model
"""
output, hn = rnn(tweet_input_embeddings, h0) # h0 is optional input, defaults to tensor of 0's of apprpriate size (num_layers, batch, hidden_size) when not provided

But what is ``output``? Well, let's inspect its shape first:

In [38]:
# output = seq_len, batch, hidden_size (output features from last layer of RNN)
print("output size: ", output.size())

output size:  torch.Size([22, 4, 50])


Here's what we need to know about ``output``:
- The first dimension in the ``output`` tensor is the ``max_seq length`` (22). 
- The second dimension is ``batch_size`` (the number of examples/sentences in our batch = 4).
- The third dimension is the ``size of nodes/units`` in our hidden layer (=50). 

Here's what we need to know about ``hn``:
- ``hn`` is a tensor of shape (num_layers, batch_size, hidden_size / number of hidden layer nodes) containing the hidden state for the last ``time step`` 
(``t = max_seq_length``).

In [39]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([1, 4, 50])


You can take the output representation for a tweet after processing the last token (t=seq_len or last timestep) and call the resulting representation as the tweet representation that "summarizes" the information present in the tweet. This tweet representation can further be used for a useful task like tweet classification (we will try out sentiment analysis later in this tutorial) by adding a classification module on top of the tweet representation.

Let us compute the final tweet representation:

In [40]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (4)
# second dimension - number of features in hidden state h_t (50, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])


## Multilayered RNN

For some applications, we may need more than one hidden layer for RNN to model the information flow. Adding more layers requires fews changes.

Firstly, we change the ``num_layers`` argument to reflect the number of layers we want during the RNN module definition.

In [41]:
"""
define the RNN module
"""
# first input - number of dimesnions for word vectors for a vector x (300, size of the word embedding)
# second input - number of nodes in hidden layer (50, size of the hidden layer)
# third input - number of recurrent layers (we set it to 1)
rnn = nn.RNN(input_size=300, hidden_size=50, num_layers=2) # input_size, hidden_size, num_layers

Similar to single layered RNN, Multilayered RNN module takes two inputs: the ``initial hidden state h0`` for each element in the batch (at ``time step t=0``) and the ``input features`` (``tweet_input_embeddings`` in our case).

Let us construct the new initial hidden state for a 2 layered RNN.

In [42]:
"""
hidden layer at time-step 0 (h_0)
"""
# first dimension - number of RNN layers (2)
# second dimension - number of examples/sentences in a batch (4)
# third dimension - number of nodes in hidden layer (50, size of the hidden layer)
h0 = torch.randn(2, 4, 50)
print("The shape as as expected: ", h0.shape)

The shape as as expected:  torch.Size([2, 4, 50])


Let us feed both the hidden representation constructed above and tweet embeddings to our RNN model.

In [43]:
"""
forward propagation over the RNN model
"""
output, hn = rnn(tweet_input_embeddings, h0) # h0 is optional input, defaults to tensor of 0's when not provided

``output`` tensor contains the output features $h_t$ from the last layer of the RNN

In [44]:
# output = seq_len, batch, hidden_size (output features from last layer of RNN)
print("output size: ", output.size())

output size:  torch.Size([22, 4, 50])


``hn`` is a tensor of shape (num_layers, batch_size, hidden_size / number of nodes in a hidden layer) containing the hidden state for last time step ``t = max_seq_len`` for the ``2 layered RNN``.

In [45]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([2, 4, 50])


Let us compute the final tweet representation:

In [46]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (4)
# second dimension - number of features in hidden state h_t (50, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])


In [47]:
tweet_output_embeddings

tensor([[-0.4750, -0.0382, -0.3033, -0.6008,  0.0755, -0.5770,  0.2852,  0.5443,
         -0.1630,  0.3962, -0.1597,  0.0470,  0.0747,  0.2451,  0.2394,  0.2970,
          0.1088,  0.1043, -0.1865,  0.4555,  0.0305, -0.4840,  0.1716, -0.0013,
         -0.6560, -0.4392, -0.0558, -0.0386,  0.2135, -0.0366, -0.2997,  0.4148,
         -0.4572,  0.1798,  0.3508, -0.1521,  0.3993, -0.2273,  0.2632,  0.4344,
         -0.6013,  0.6624, -0.1212, -0.2163, -0.3276,  0.3365,  0.3889,  0.2173,
          0.4296, -0.5430],
        [-0.5174, -0.0487, -0.3424, -0.5970,  0.0584, -0.5848,  0.3159,  0.5250,
         -0.1840,  0.4092, -0.2140,  0.0445,  0.0902,  0.2211,  0.2354,  0.2855,
          0.1305,  0.1047, -0.1948,  0.4687,  0.0277, -0.4609,  0.1987, -0.0165,
         -0.6700, -0.4564, -0.0676, -0.0252,  0.2396, -0.0109, -0.2977,  0.3746,
         -0.4520,  0.1945,  0.3467, -0.1671,  0.3785, -0.2013,  0.2746,  0.4617,
         -0.5988,  0.6594, -0.1220, -0.2084, -0.3203,  0.3613,  0.4038,  0.2139,


## RNN for Sentiment Analysis

In this section we will implement RNN for classifying the sentiment of the tweet (same task used in our previous feedforward neural networks tutorial).

We will pick up most of the functions from our feedforward neural networks code:

In [48]:
# all the necessary imports
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim

# set the seed
manual_seed = 123
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
  torch.cuda.manual_seed(manual_seed)

# hyperparameters
MAX_EPOCHS = 5
LEARNING_RATE = 0.3
NUM_CLASSES = 3
EMBEDDING_SIZE = 300

Now we can define the full RNN model:

In [49]:
"""
create a model for RNN
"""
class RNNmodel(nn.Module):
  
  def __init__(self, embedding_size, vocab_size, output_size, hidden_size, num_layers):
    # In the constructor we define the layers for our model
    super(RNNmodel, self).__init__()
    # word embedding lookup table
    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size, sparse=True)
    # core RNN module
    self.rnn_layer = nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers) 
    # activation function
    self.activation_fn = nn.ReLU()
    # classification related modules
    self.linear_layer = nn.Linear(hidden_size, output_size) 
    self.softmax_layer = nn.LogSoftmax(dim=0)
  
  def forward(self, x):
    # In the forward function we define the forward propagation logic
    out = self.embedding(x)
    out, _ = self.rnn_layer(out) # since we are not feeding h_0 explicitly, h_0 will be initialized to zeros by default
    # classify based on the hidden representation after RNN processes the last token
    out = out[-1]
    out = self.activation_fn(out)
    out = self.linear_layer(out)
    out = self.softmax_layer(out) # accepts 2D or more dimensional inputs
    return out

Some additional hyperparameters for RNN

In [50]:
# hyperparameters of RNN
HIDDEN_SIZE = 50
NUM_LAYERS = 2

Rest of the pipeline looks similar to our feedforward neural networks code (except that we are using **torchtext** instead of **DataLoader**):

In [56]:
from sklearn.metrics import accuracy_score
def train(loader):
    total_loss = 0.0
    # iterate throught the data loader
    num_sample = 0
    for batch in loader:
        # load the current batch
        batch_input = batch.tweet
        batch_output = batch.label
        
        batch_input = batch_input.to(device)
        batch_output = batch_output.to(device)
        # forward propagation
        # pass the data through the model
        model_outputs = model(batch_input)
        # compute the loss
        cur_loss = criterion(model_outputs, batch_output)
        total_loss += cur_loss.item()

        # backward propagation (compute the gradients and update the model)
        # clear the buffer
        optimizer.zero_grad()
        # compute the gradients
        cur_loss.backward()
        # update the weights
        optimizer.step()

        num_sample += batch_output.shape[0]
    return total_loss/num_sample

# evaluation logic based on classification accuracy
def evaluate(loader):
    all_pred=[]
    all_label = []
    with torch.no_grad(): # impacts the autograd engine and deactivate it. reduces memory usage and speeds up computation
        for batch in loader:
             # load the current batch
            batch_input = batch.tweet
            batch_output = batch.label

            batch_input = batch_input.to(device)
            # forward propagation
            # pass the data through the model
            model_outputs = model(batch_input)
            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(model_outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(batch_output)
            
    accuracy = accuracy_score(all_label, all_pred)
    return accuracy

Let us define the RNN model.

In [57]:
# define the model
model = RNNmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, HIDDEN_SIZE, NUM_LAYERS) 
model.to(device)
# define the loss function (last node of the graph)
criterion = nn.NLLLoss()

**We need to create a new directory 'ckpt/' to store our model checkpoint.**

In [54]:
import os
os.mkdir("./ckpt")

**Let us perform the training. We will save our model and optimizer at end of each epoch.**


You can find more information of saving and loading model [here](https://pytorch.org/tutorials/beginner/saving_loading_models.html).

In [63]:
# create an instance of SGD with required hyperparameters
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

# start the training
for epoch in range(MAX_EPOCHS):
    # train the model for one pass over the data
    train_loss = train(train_iter)  
    # compute the training accuracy
    train_acc = evaluate(train_iter)
    # compute the validation accuracy
    val_acc = evaluate(val_iter)
    
    # print the loss for every epoch
    print('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {:.4f}, Validation Accuracy: {:.4f}'.format(epoch+1, MAX_EPOCHS, train_loss, train_acc, val_acc))
    
    # save model, optimizer, and number of epoch to a dictionary
    model_save = {
            'epoch': epoch,  # number of epoch
            'model_state_dict': model.state_dict(), # model parameters 
            'optimizer_state_dict': optimizer.state_dict(), # save optimizer 
            'loss': train_loss # training loss
            }
    
    # use torch.save to store 
    torch.save(model_save, "./ckpt/model_{}.pt".format(epoch))

Epoch [1/5], Loss: 0.3452, Training Accuracy: 0.4048, Validation Accuracy: 0.3392
Epoch [2/5], Loss: 0.3448, Training Accuracy: 0.4015, Validation Accuracy: 0.3972
Epoch [3/5], Loss: 0.3449, Training Accuracy: 0.3438, Validation Accuracy: 0.3657
Epoch [4/5], Loss: 0.3448, Training Accuracy: 0.3568, Validation Accuracy: 0.3442
Epoch [5/5], Loss: 0.3450, Training Accuracy: 0.3432, Validation Accuracy: 0.3847


We trained the network only for 5 epochs, but it already overfits on validation set after epoch 2. 
In the coming sessions, we will look at methods to ``regularize`` the network (this will help us deal with overfitting).

**Load model checkpoint** 

When we have a trained model checkpint, we can load it using `torch.load()`

In [66]:
# define a new model
model2 = RNNmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, HIDDEN_SIZE, NUM_LAYERS) 
# load checkpoint 
checkpoint = torch.load("./ckpt/model_1.pt")
# assign the parameters of checkpoint to this new model
model2.load_state_dict(checkpoint['model_state_dict'])
model2.to(device)

print(model2)

RNNmodel(
  (embedding): Embedding(4876, 300, sparse=True)
  (rnn_layer): RNN(300, 50, num_layers=2)
  (activation_fn): ReLU()
  (linear_layer): Linear(in_features=50, out_features=3, bias=True)
  (softmax_layer): LogSoftmax()
)


## GRUs

Gated Recurrent Units (GRUs) are a variant of RNNs that use more complex units for activation. They are created to have more persistent memory thereby making them easier for RNNs to capture long-term dependencies. To learn the theory behind GRUs, we recommend: https://github.com/UBC-NLP/dlnlp2019/blob/master/slides/RNN.pdf 

GRU is defined by ``torch.nn.GRU`` module and its documentation can be fetched [here](https://pytorch.org/docs/stable/nn.html#torch.nn.GRU). Now let us define the GRU module.

In [313]:
"""
define the GRU module
"""
# first input - number of word vector dimensions/embeddings
# second input - number of nodes in hidden layer (50, size of the hidden layer)
# third input - number of recurrent layers (2)
gru_rnn = nn.GRU(input_size=300, hidden_size=50, num_layers=2) # input_size, hidden_size, num_layers

Similar to RNN, GRU module takes two inputs: *the initial hidden state for each element in the batch* (t=0) and the *input features* (``tweet_input_embeddings`` in our case).

Let us feed both the initial hidden state and tweet embeddings to our GRU model.

In [314]:
"""
forward propagation over the GRU model
"""
output, hn = gru_rnn(tweet_input_embeddings, h0) # h0 is optional input, defaults to tensor of 0's when not provided

``output`` tensor contqains the output features $h_t$ from the last layer of the GRU

In [315]:
# output = seq_len, batch, hidden_size (output features from last layer of GRU)
print("output size: ", output.size())

output size:  torch.Size([20, 4, 50])


``hn`` is a tensor of shape (num_layers, batch_size, hidden_size / number of nodes in a hidden layer) containing the hidden state for last time step ``t = max_seq_len`` for the ``2 layered RNN``.

In [316]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([2, 4, 50])


Similar to RNN, you can compute the final tweet representation (representation from last hidden state for each tweet) as follows.

In [317]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (5)
# second dimension - number of features in hidden state h_t (20, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])


## LSTMs

Long short-term memory (LSTMs) are a variant of RNNs that use more complex units for activation. Similar to the spirit of GRU, they are created to have more persistent memory thereby making them easier for RNNs to capture long-term dependencies. To learn the theory behind GRUs, we recommend: https://github.com/UBC-NLP/dlnlp2019/blob/master/slides/RNN.pdf 

LSTM is defined by ``torch.nn.LSTM`` module and its documentation can be fetched [here](https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM). Now let us define the LSTM module.

In [322]:
"""
define the LSTM module
"""
# first input - number of features in x (300, size of the word embedding)
# second input - number of number of nodes in a hidden layer (50)
# third input - number of recurrent layers (2)
lstm_rnn = nn.LSTM(input_size=300, hidden_size=50, num_layers=2) # input_size, hidden_size, num_layers

Unlike RNN and GRU, LSTM module takes three inputs: the initial hidden state for each element in the batch (t=0), the input features (tweet_input_embeddings in our case) and initial cell state for each element in the batch.

Let us construct the initial cell state (this construction is similar to that of initial hidden state)

In [331]:
"""
cell state at time-step 0 (h_0)
"""
# first dimension - number of LSTM layers (2)
# second dimension - batch_size (# of tweets/examples/sentences)
# third dimension - hidden_size / number of nodes in a hidden layer (50)
c0 = torch.randn(2, 4, 50)

Let us feed the initial hidden state, initial cell state and tweet embeddings to our LSTM model.

In [327]:
"""
forward propagation over the LSTM model
"""
output, (hn, cn) = lstm_rnn(tweet_input_embeddings, (h0, c0)) # h0 and c0 is optional input, defaults to tensor of 0's when not provided

``output`` tensor contains the output features $h_t$ from the last layer of the LSTM

In [328]:
# output = seq_len, batch_size, hidden_size (output features from last layer of LSTM)
print("output size: ", output.size())

output size:  torch.Size([20, 4, 50])


``hn`` is a tensor of shape (num_layers, batch, hidden_size) containing the hidden state for t = seq_len

In [39]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([2, 5, 20])


``cn`` is a tensor of shape (num_layers, batch, hidden_size) containing the cell state for t = seq_len.

In [329]:
# c_n = num_layers, batch_size, hidden_size (cell state for t=seq_len or cell state at last timestep)
print("last cell state size: ", hn.size())

last cell state size:  torch.Size([2, 4, 50])


Similar to RNN and GRU, you can compute the final tweet representation (representation from last hidden state for each tweet) as follows.

In [330]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (4)
# second dimension - number of features in hidden state h_t (50, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])
